# 3교시: Streamlit으로 웹앱 만들기

## 실습 목표
- Streamlit 기본 사용법 익히기
- LLM API와 연동한 챗봇 UI 만들기
- (Colab) ngrok을 이용한 실행 방법

## 사용 모델
- **Google Gemini 2.0 Flash** (무료)

---
## 1. 환경 설정

In [ ]:
# 필요한 패키지 설치
!pip install streamlit google-generativeai python-dotenv -q

In [ ]:
# Colab에서 실행하는 경우 추가 설치
# !pip install pyngrok -q

---
## 2. Streamlit 기본 앱 만들기

### 2.1 Hello World 앱

In [ ]:
%%writefile hello_streamlit.py
"""
첫 번째 Streamlit 앱
실행: streamlit run hello_streamlit.py
"""
import streamlit as st

# 페이지 설정
st.set_page_config(
    page_title="첫 번째 앱",
    page_icon="🎉",
    layout="centered"
)

# 제목
st.title("Hello, Streamlit! 🎈")

# 텍스트 출력
st.write("파이썬으로 웹앱을 만들어봅시다!")

# 마크다운
st.markdown("""
### Streamlit의 장점
- **쉬움**: Python만 알면 됨
- **빠름**: 몇 분만에 완성
- **예쁨**: 기본 스타일이 깔끔함
""")

# 이미지 (URL로도 가능)
st.image("https://streamlit.io/images/brand/streamlit-logo-primary-colormark-darktext.png", width=300)

### 2.2 인터랙티브 데모 앱

In [ ]:
%%writefile interactive_demo.py
"""
인터랙티브 위젯 데모
실행: streamlit run interactive_demo.py
"""
import streamlit as st
import pandas as pd

st.set_page_config(page_title="인터랙티브 데모", page_icon="🎮")

st.title("Streamlit 위젯 데모 🎮")

# 사이드바
with st.sidebar:
    st.header("설정")
    theme = st.selectbox("테마 선택", ["밝은 테마", "어두운 테마"])
    st.divider()
    st.info("이 앱은 Streamlit 위젯을 시연합니다.")

# 탭 구성
tab1, tab2, tab3 = st.tabs(["📝 입력", "📊 데이터", "🎨 스타일"])

with tab1:
    st.header("입력 위젯")
    
    # 텍스트 입력
    name = st.text_input("이름을 입력하세요", placeholder="홍길동")
    if name:
        st.write(f"안녕하세요, **{name}**님! 👋")
    
    # 숫자 슬라이더
    age = st.slider("나이를 선택하세요", 1, 100, 20)
    st.write(f"선택한 나이: {age}세")
    
    # 라디오 버튼
    hobby = st.radio(
        "취미를 선택하세요",
        ["독서 📚", "운동 🏃", "게임 🎮", "음악 🎵"],
        horizontal=True
    )
    
    # 버튼
    if st.button("정보 확인", type="primary"):
        if name:
            st.success(f"{name}님은 {age}세이고, 취미는 {hobby}입니다!")
        else:
            st.warning("이름을 입력해주세요.")

with tab2:
    st.header("데이터 표시")
    
    # 샘플 데이터
    df = pd.DataFrame({
        'LLM 모델': ['GPT-4', 'Claude 3', 'Gemini', 'LLaMA 3'],
        '회사': ['OpenAI', 'Anthropic', 'Google', 'Meta'],
        '출시년도': [2023, 2024, 2024, 2024],
        '인기도': [95, 88, 85, 75]
    })
    
    # 데이터프레임 표시
    st.dataframe(df, use_container_width=True)
    
    # 차트
    st.bar_chart(df.set_index('LLM 모델')['인기도'])
    
    # 메트릭
    col1, col2, col3 = st.columns(3)
    col1.metric("총 모델 수", len(df))
    col2.metric("평균 인기도", f"{df['인기도'].mean():.1f}")
    col3.metric("최신 모델", df.loc[df['출시년도'].idxmax(), 'LLM 모델'])

with tab3:
    st.header("알림 스타일")
    
    st.success("✅ 성공 메시지입니다!")
    st.info("ℹ️ 정보 메시지입니다.")
    st.warning("⚠️ 경고 메시지입니다.")
    st.error("❌ 에러 메시지입니다.")
    
    # 코드 블록
    st.code("""
import streamlit as st
st.write("Hello, World!")
    """, language="python")
    
    # 확장 영역
    with st.expander("더 보기"):
        st.write("숨겨진 내용이 여기에 있습니다!")

---
## 3. LLM 챗봇 앱 만들기 (Gemini 버전)

### 3.1 기본 챗봇

In [ ]:
%%writefile chatbot_app.py
"""
Gemini 챗봇 앱
실행: streamlit run chatbot_app.py
"""
import streamlit as st
import google.generativeai as genai

# 페이지 설정
st.set_page_config(
    page_title="AI 챗봇",
    page_icon="🤖",
    layout="centered"
)

st.title("AI 챗봇 🤖")
st.caption("Google Gemini와 대화해보세요! (무료)")

# 사이드바 - API Key 입력 및 설정
with st.sidebar:
    st.header("⚙️ 설정")
    
    # API Key 입력
    api_key = st.text_input(
        "Google API Key",
        type="password",
        placeholder="AIza...",
        help="https://aistudio.google.com/app/apikey 에서 발급"
    )
    
    # 모델 선택
    model_name = st.selectbox(
        "모델 선택",
        ["gemini-2.5-flash-lite", "gemini-1.5-flash", "gemini-1.5-pro"]
    )
    
    # Temperature 설정
    temperature = st.slider(
        "Temperature",
        min_value=0.0,
        max_value=1.0,
        value=0.7,
        step=0.1,
        help="높을수록 창의적, 낮을수록 일관적"
    )
    
    st.divider()
    
    # 대화 초기화 버튼
    if st.button("대화 초기화", type="secondary", use_container_width=True):
        st.session_state.messages = []
        st.session_state.chat = None
        st.rerun()

# 메시지 기록 초기화
if "messages" not in st.session_state:
    st.session_state.messages = []

if "chat" not in st.session_state:
    st.session_state.chat = None

# 이전 메시지 표시
for message in st.session_state.messages:
    with st.chat_message(message["role"]):
        st.markdown(message["content"])

# 새 메시지 입력
if prompt := st.chat_input("메시지를 입력하세요..."):
    # API Key 확인
    if not api_key:
        st.error("API Key를 입력해주세요!")
        st.stop()
    
    # 사용자 메시지 추가
    st.session_state.messages.append({"role": "user", "content": prompt})
    with st.chat_message("user"):
        st.markdown(prompt)
    
    # AI 응답 생성
    with st.chat_message("assistant"):
        with st.spinner("생각 중..."):
            try:
                # Gemini 설정
                genai.configure(api_key=api_key)
                
                # 채팅 세션 생성 또는 재사용
                if st.session_state.chat is None:
                    model = genai.GenerativeModel(
                        model_name,
                        generation_config=genai.GenerationConfig(
                            temperature=temperature
                        ),
                        system_instruction="당신은 친절하고 도움이 되는 AI 어시스턴트입니다. 한국어로 대화합니다."
                    )
                    st.session_state.chat = model.start_chat(history=[])
                
                # 응답 생성
                response = st.session_state.chat.send_message(prompt)
                answer = response.text
                st.markdown(answer)
                
                # 응답 저장
                st.session_state.messages.append({"role": "assistant", "content": answer})
                
            except Exception as e:
                st.error(f"오류 발생: {str(e)}")

### 3.2 스트리밍 챗봇 (고급)

In [ ]:
%%writefile chatbot_streaming.py
"""
스트리밍 Gemini 챗봇 앱
실행: streamlit run chatbot_streaming.py
"""
import streamlit as st
import google.generativeai as genai

st.set_page_config(page_title="스트리밍 챗봇", page_icon="⚡")

st.title("스트리밍 챗봇 ⚡")
st.caption("실시간으로 응답을 확인하세요! (Gemini 무료)")

# 사이드바
with st.sidebar:
    api_key = st.text_input("Google API Key", type="password")
    model_name = st.selectbox("모델", ["gemini-2.5-flash-lite", "gemini-1.5-flash"])
    
    # 시스템 프롬프트 커스터마이징
    system_prompt = st.text_area(
        "시스템 프롬프트",
        value="당신은 친절한 AI 어시스턴트입니다. 한국어로 답변합니다.",
        height=100
    )
    
    if st.button("대화 초기화"):
        st.session_state.messages = []
        st.session_state.chat = None
        st.rerun()

# 메시지 초기화
if "messages" not in st.session_state:
    st.session_state.messages = []

if "chat" not in st.session_state:
    st.session_state.chat = None

# 이전 메시지 표시
for message in st.session_state.messages:
    with st.chat_message(message["role"]):
        st.markdown(message["content"])

# 새 메시지 처리
if prompt := st.chat_input("메시지 입력..."):
    if not api_key:
        st.error("API Key를 입력해주세요!")
        st.stop()
    
    # 사용자 메시지
    st.session_state.messages.append({"role": "user", "content": prompt})
    with st.chat_message("user"):
        st.markdown(prompt)
    
    # 스트리밍 응답
    with st.chat_message("assistant"):
        try:
            genai.configure(api_key=api_key)
            
            # 채팅 세션 생성
            if st.session_state.chat is None:
                model = genai.GenerativeModel(
                    model_name,
                    system_instruction=system_prompt
                )
                st.session_state.chat = model.start_chat(history=[])
            
            # 스트리밍 응답
            response = st.session_state.chat.send_message(prompt, stream=True)
            
            # 스트리밍 출력
            response_text = st.write_stream(
                (chunk.text for chunk in response)
            )
            
            # 응답 저장
            st.session_state.messages.append({"role": "assistant", "content": response_text})
            
        except Exception as e:
            st.error(f"오류: {str(e)}")

---
## 4. 로컬에서 실행하기

터미널에서 다음 명령어로 실행하세요:

```bash
# 기본 앱 실행
streamlit run hello_streamlit.py

# 인터랙티브 데모
streamlit run interactive_demo.py

# 챗봇 앱
streamlit run chatbot_app.py

# 스트리밍 챗봇
streamlit run chatbot_streaming.py
```

---
## 5. Google Colab에서 실행하기

In [ ]:
# Colab에서만 실행하세요!
# ngrok 설치
# !pip install pyngrok -q

# ngrok 설정 (https://ngrok.com 에서 토큰 발급)
# from pyngrok import ngrok
# ngrok.set_auth_token("YOUR_NGROK_TOKEN")  # 본인 토큰으로 교체

# 터널 생성
# public_url = ngrok.connect(8501)
# print(f"\n접속 URL: {public_url}")

# Streamlit 실행 (백그라운드)
# !streamlit run chatbot_app.py &

---
## 6. 실습 과제

### 과제 1: 나만의 챗봇 페르소나 만들기

아래 코드를 수정하여 특별한 페르소나를 가진 챗봇을 만들어보세요.

In [ ]:
%%writefile my_persona_chatbot.py
"""
나만의 페르소나 챗봇 (Gemini 버전)
TODO: 시스템 프롬프트를 수정하여 독특한 캐릭터를 만들어보세요!
"""
import streamlit as st
import google.generativeai as genai

st.set_page_config(page_title="나만의 챗봇", page_icon="🌟")

# TODO: 챗봇 이름과 설명을 수정하세요
st.title("[챗봇 이름] 🌟")
st.caption("[챗봇 설명]")

# TODO: 시스템 프롬프트를 수정하여 챗봇의 성격을 정의하세요
# 예시: 
# - "당신은 조선시대 선비입니다. 고풍스러운 말투로 대화합니다."
# - "당신은 열정적인 요리사입니다. 모든 것을 음식에 비유합니다."
# - "당신은 고양이입니다. 문장 끝에 '냥'을 붙입니다."
SYSTEM_PROMPT = """
여기에 페르소나를 정의하세요!
"""

with st.sidebar:
    api_key = st.text_input("Google API Key", type="password")
    if st.button("초기화"):
        st.session_state.messages = []
        st.session_state.chat = None
        st.rerun()

if "messages" not in st.session_state:
    st.session_state.messages = []

if "chat" not in st.session_state:
    st.session_state.chat = None

for msg in st.session_state.messages:
    with st.chat_message(msg["role"]):
        st.markdown(msg["content"])

if prompt := st.chat_input("대화를 시작하세요..."):
    if not api_key:
        st.error("API Key 필요!")
        st.stop()
    
    st.session_state.messages.append({"role": "user", "content": prompt})
    with st.chat_message("user"):
        st.markdown(prompt)
    
    with st.chat_message("assistant"):
        genai.configure(api_key=api_key)
        
        if st.session_state.chat is None:
            model = genai.GenerativeModel(
                "gemini-2.5-flash-lite",
                system_instruction=SYSTEM_PROMPT
            )
            st.session_state.chat = model.start_chat(history=[])
        
        response = st.session_state.chat.send_message(prompt)
        answer = response.text
        st.markdown(answer)
        st.session_state.messages.append({"role": "assistant", "content": answer})

### 과제 2: 추가 기능 넣기

다음 중 하나 이상의 기능을 챗봇에 추가해보세요:

1. 대화 내용 다운로드 버튼
2. 다크/라이트 테마 전환
3. 대화 기록 개수 제한 (최근 10개만 유지)
4. 특정 키워드 감지 시 알림 표시

In [ ]:
# 힌트: 대화 내용 다운로드 기능
import json

# session_state.messages를 JSON으로 변환
# messages_json = json.dumps(
#     st.session_state.get("messages", []), 
#     ensure_ascii=False, 
#     indent=2
# )

# 다운로드 버튼 예시
# st.download_button(
#     label="대화 내용 다운로드",
#     data=messages_json,
#     file_name="chat_history.json",
#     mime="application/json"
# )

---
## 7. 유용한 리소스

### Streamlit
- 공식 문서: https://docs.streamlit.io
- 치트 시트: https://docs.streamlit.io/library/cheatsheet
- 갤러리: https://streamlit.io/gallery

### Google Gemini
- API Key 발급: https://aistudio.google.com/app/apikey
- 공식 문서: https://ai.google.dev/gemini-api/docs
- Python SDK: https://github.com/google/generative-ai-python

---
## 정리

### 오늘 배운 내용
1. **Streamlit 기초**: 텍스트, 위젯, 레이아웃
2. **Session State**: 상태 관리로 대화 기록 유지
3. **Chat UI**: `st.chat_message`, `st.chat_input`
4. **API 연동**: Gemini API와 Streamlit 통합
5. **스트리밍**: 실시간 응답 출력

### 사용 API
- **Google Gemini 2.0 Flash** (무료)
- API Key: https://aistudio.google.com/app/apikey
